In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.spatial import distance
from scipy.stats import ttest_ind

import scipy.cluster as cluster

from statsmodels.stats.multitest import multipletests

from sklearn import metrics

In [ ]:
pheno_et = pd.read_csv('pheno_et.csv')
pheno_et.set_index(['EID'], inplace=True)
pheno_et = pheno_et.astype(np.float64)

In [ ]:
x = np.load('et_time_series_x.npy')
y = np.load('et_time_series_y.npy')

In [ ]:
et = np.stack((x, y), axis=2)

# Euclidean distance

In [ ]:
dis = np.zeros((et.shape[0], et.shape[0]))

min_e = np.inf
for i in range(et.shape[0] - 1):
    for j in range(i + 1, et.shape[0]):
        e = np.mean(np.sqrt((et[i,:,0] - et[j,:,0])**2 + (et[i,:,1] - et[j,:,1])**2))
        if e < min_e:
            min_e = e
        dis[i,j] = e
        dis[j,i] = e

In [ ]:
plt.figure(figsize=(10,10))
im = plt.imshow(dis, cmap='jet', vmin=min_e, vmax=125)
plt.colorbar(im)

In [ ]:
cd = distance.squareform(dis) # since inputting dense distance matrix into linkage function, must convert to 1D condensed distance vector

In [ ]:
Z = cluster.hierarchy.linkage(cd, method='ward')

In [ ]:
plt.figure(figsize=(11,11))
dn = cluster.hierarchy.dendrogram(Z)
plt.xticks([])

In [ ]:
clusters = cluster.hierarchy.fcluster(Z, 500, criterion='distance')

In [ ]:
u, c = np.unique(clusters, return_counts=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(35,20))

for i in range(u.size):
    ax[0].plot(np.mean(x[clusters == u[i]], axis=0))
    ax[1].plot(np.mean(y[clusters == u[i]], axis=0), label='%d; N=%d' % (u[i], c[i]))
ax[0].set_title('x-direction mean time series', fontsize=18, fontweight='bold')
ax[1].set_title('y-direction mean time series', fontsize=18, fontweight='bold')
ax[1].legend(loc=0, fontsize=18)

In [ ]:
dis_clust = dis[np.argsort(clusters),:][:,np.argsort(clusters)] # sort distance matrix by cluster assignments

In [ ]:
plt.figure(figsize=(10,10))
im = plt.imshow(dis_clust, cmap='jet', vmin=min_e, vmax=160)
plt.colorbar(im)

In [ ]:
# compile behavioral data for each cluster
pheno_c = []
for i in np.unique(clusters):
    pheno_c.append(pheno_et.iloc[clusters == i,:-1])

In [ ]:
# calculate group differences for each measure
pvals = np.zeros((np.unique(clusters).size, np.unique(clusters).size, 4))
for i in range(np.unique(clusters).size - 1):
    for j in range(i + 1, np.unique(clusters).size):
        for k in range(4):
            p = ttest_ind(pheno_c[i].iloc[:,k], pheno_c[j].iloc[:,k], equal_var=False)[1]
            pvals[i,j,k] = p
            pvals[j,i,k] = p

In [ ]:
# FDR correction
p_corr = np.zeros(pvals.shape)
nmu = np.triu_indices(pvals.shape[0], 1)
for i in range(pvals.shape[2]):
    nm = np.triu_indices(np.unique(clusters).size, 1)
    p_unc = pvals[...,i][nm[0],nm[1]]
    res = multipletests(p_unc, method='fdr_bh')
    p_corr[nmu[0],nmu[1],i] = res[1]
    p_corr[...,i] += p_corr[...,i].T

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(11,11))
ax_flat = ax.flat
scores = pheno_et.columns.tolist()[:-1]
for i in range(p_corr.shape[2]):
    im = ax_flat[i].imshow(p_corr[...,i], cmap='jet', vmax=0.05)
    ax_flat[i].set_title(scores[i], fontsize=18)
    ax_flat[i].set_xticks(np.arange(p_corr.shape[0]))
    ax_flat[i].set_yticks(np.arange(p_corr.shape[0]))
    ax_flat[i].set_xticklabels(np.arange(p_corr.shape[0]) + 1)
    ax_flat[i].set_yticklabels(np.arange(p_corr.shape[0]) + 1)
cbar = fig.colorbar(im, ax=ax.ravel().tolist(), shrink=0.95)

In [ ]:
pheno_et['clusters'] = clusters

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(30,15))
sns.violinplot(x='clusters', y='SCQ_Total', data=pheno_et, ax=ax[0])
sns.violinplot(x='clusters', y='SRS_Total_T', data=pheno_et, ax=ax[1])
ax[0].set_title('SCQ_Total', fontsize=18, fontweight='bold')
ax[1].set_title('SRS_Total_T', fontsize=18, fontweight='bold')

# Cosine distance

In [ ]:
dis = np.zeros((et.shape[0], et.shape[0]))

# originally, origin of ET data is top left corner--this does not suit cosine distance well, so need to center data at center of screen
et[...,0] -= 400
et[...,1] -= 300

min_cos = np.inf
for i in range(et.shape[0] - 1):
    for j in range(i + 1, et.shape[0]):
        cos = distance.cosine(np.concatenate((et[i,:,0], et[i,:,1])), np.concatenate((et[j,:,0], et[j,:,1])))
        if cos < min_cos:
            min_cos = cos
        dis[i,j] = cos
        dis[j,i] = cos

In [ ]:
plt.figure(figsize=(10,10))
im = plt.imshow(dis, cmap='jet', vmin=min_cos, vmax=1.2)
plt.colorbar(im)

In [ ]:
# bootstrapping
niter = 1000
chunk = int(et.shape[0] * 0.9)

# create consensus matrix for 4 different cluster amounts in dendrogram
consensus_mats = [
    np.zeros((et.shape[0], et.shape[0])),
    np.zeros((et.shape[0], et.shape[0])),
    np.zeros((et.shape[0], et.shape[0])),
    np.zeros((et.shape[0], et.shape[0]))
]

for i in range(niter):
    idx = np.random.choice(et.shape[0], chunk, replace=False)
    idx.sort()
    dis_boot = dis[idx][:,idx]
    cd = distance.squareform(dis_boot)
    Z = cluster.hierarchy.linkage(cd, method='ward')
    for c in range(2, 6):
        cm = consensus_mats[c - 2]
        clusters = cluster.hierarchy.fcluster(Z, c, criterion='maxclust') # 'maxclust' cuts dendrogram so that it creates c optimal clusters
        
        # in order to use the clever method below, preserve unselected indices as 0 (cluster labels start at 1)
        ct = np.zeros((et.shape[0]),)
        ct[idx] = clusters
        
        ####### extremely clever way of creating consensus matrix -- taken from Aki's PyBASC #######
        ct = ct[:,np.newaxis]
        cm += (np.dot(ct**-1., ct.T) == 1).astype(np.float64)
        ############################################################################################
        
for mat in consensus_mats:
    mat /= niter

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))
ax = axes.flat
for i in range(len(ax)):
    im = ax[i].imshow(consensus_mats[i], cmap='Reds')
    ax[i].set_title('Dendrogram cut at %d clusters' % (i + 2))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))
ax = axes.flat
for i in range(len(consensus_mats)):
    mat = np.copy(consensus_mats[i])
    mat = 1 - mat
    np.fill_diagonal(mat, 0)
    cd = distance.squareform(mat)
    Z = cluster.hierarchy.linkage(cd, method='ward')
    dn = cluster.hierarchy.dendrogram(Z, ax=ax[i])
    ax[i].set_xticks([])
    ax[i].set_title('%d-cluster consensus matrix' % (i + 2))

In [ ]:
silhouettes = np.zeros((4,3))
for i in range(len(consensus_mats)):
    mat = np.copy(consensus_mats[i])
    mat = 1 - mat
    np.fill_diagonal(mat, 0)
    cd = distance.squareform(mat)
    Z = cluster.hierarchy.linkage(cd, method='ward')
    for c in range(2, 5):
        clusters = cluster.hierarchy.fcluster(Z, c, criterion='maxclust')
        sil = metrics.silhouette_score(dis, clusters, metric='precomputed')
        silhouettes[i,c - 2] = sil

In [ ]:
silhouettes

In [ ]:
i, c = np.unravel_index(np.argmax(silhouettes, axis=None), silhouettes.shape) # return cell that has highest silhouette score
mat = np.copy(consensus_mats[i])
mat = 1 - mat # make into distance matrix
np.fill_diagonal(mat, 0)
cd = distance.squareform(mat)
Z = cluster.hierarchy.linkage(cd, method='ward')
clusters = cluster.hierarchy.fcluster(Z, c + 2, criterion='maxclust')

In [ ]:
np.unique(clusters, return_counts=True)

In [ ]:
pheno_c = []
for i in np.unique(clusters):
    pheno_c.append(pheno_et.iloc[clusters == i,:-1])

In [ ]:
pvals = np.zeros((np.unique(clusters).size, np.unique(clusters).size, 4))
for i in range(np.unique(clusters).size - 1):
    for j in range(i + 1, np.unique(clusters).size):
        for k in range(4):
            p = ttest_ind(pheno_c[i].iloc[:,k], pheno_c[j].iloc[:,k], equal_var=False)[1]
            pvals[i,j,k] = p
            pvals[j,i,k] = p

In [ ]:
p_corr = np.zeros(pvals.shape)
nmu = np.triu_indices(pvals.shape[0], 1)
for i in range(pvals.shape[2]):
    nm = np.triu_indices(np.unique(clusters).size, 1)
    p_unc = pvals[...,i][nm[0],nm[1]]
    res = multipletests(p_unc, method='fdr_bh')
    p_corr[nmu[0],nmu[1],i] = res[1]
    p_corr[...,i] += p_corr[...,i].T

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(11,11))
ax_flat = ax.flat
scores = pheno_et.columns.tolist()[:-1]
for i in range(p_corr.shape[2]):
    im = ax_flat[i].imshow(p_corr[...,i], cmap='jet', vmax=0.05)
    ax_flat[i].set_title(scores[i], fontsize=18)
    ax_flat[i].set_xticks(np.arange(p_corr.shape[0]))
    ax_flat[i].set_yticks(np.arange(p_corr.shape[0]))
    ax_flat[i].set_xticklabels(np.arange(p_corr.shape[0]) + 1)
    ax_flat[i].set_yticklabels(np.arange(p_corr.shape[0]) + 1)
cbar = fig.colorbar(im, ax=ax.ravel().tolist(), shrink=0.95)

# Concordance correlation coefficient

In [ ]:
et[...,0] += 400
et[...,1] += 300

In [ ]:
def CCD(x1, y1, x2, y2):
    """
    Calculate modified concordance correlation distance for 2D data
    """
    
    data = np.stack((x1, x2, y1, y2), axis=0)
    means = np.mean(data, axis=1)
    vars_x = np.cov(x1, x2)
    vars_y = np.cov(y1, y2)
    
    ccc = 2 * (vars_x[0,1] + vars_y[0,1]) / (vars_x[0,0] + vars_x[1,1] + (means[0] - means[1])**2 + vars_y[0,0] + vars_y[1,1] + (means[2] - means[3])**2)
    return 1 - ccc

In [ ]:
dis = np.zeros((et.shape[0], et.shape[0]))

min_ccd = np.inf
for i in range(et.shape[0] - 1):
    for j in range(i + 1, et.shape[0]):
        ccd = CCD(et[i,:,0], et[i,:,1], et[j,:,0], et[j,:,1])
        if ccd < min_ccd:
            min_ccd = ccd
        dis[i,j] = ccd
        dis[j,i] = ccd

In [ ]:
plt.figure(figsize=(10,10))
im = plt.imshow(dis, cmap='jet', vmin=min_ccd)
plt.colorbar(im)

In [ ]:
# bootstrapping
niter = 1000
chunk = int(et.shape[0] * 0.9)

# create consensus matrix for 4 different cluster amounts in dendrogram
consensus_mats = [
    np.zeros((et.shape[0], et.shape[0])),
    np.zeros((et.shape[0], et.shape[0])),
    np.zeros((et.shape[0], et.shape[0])),
    np.zeros((et.shape[0], et.shape[0]))
]

for i in range(niter):
    idx = np.random.choice(et.shape[0], chunk, replace=False)
    idx.sort()
    dis_boot = dis[idx][:,idx]
    cd = distance.squareform(dis_boot)
    Z = cluster.hierarchy.linkage(cd, method='ward')
    for c in range(2, 6):
        cm = consensus_mats[c - 2]
        clusters = cluster.hierarchy.fcluster(Z, c, criterion='maxclust') # 'maxclust' cuts dendrogram so that it creates c optimal clusters
        
        # in order to use the clever method below, preserve unselected indices as 0 (cluster labels start at 1)
        ct = np.zeros((et.shape[0]),)
        ct[idx] = clusters
        
        ####### extremely clever way of creating consensus matrix -- taken from Aki's PyBASC #######
        ct = ct[:,np.newaxis]
        cm += (np.dot(ct**-1., ct.T) == 1).astype(np.float64)
        ############################################################################################
        
for mat in consensus_mats:
    mat /= niter

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))
ax = axes.flat
for i in range(len(ax)):
    im = ax[i].imshow(consensus_mats[i], cmap='Reds')
    ax[i].set_title('Dendrogram cut at %d clusters' % (i + 2))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))
ax = axes.flat
for i in range(len(consensus_mats)):
    mat = np.copy(consensus_mats[i])
    mat = 1 - mat
    np.fill_diagonal(mat, 0)
    cd = distance.squareform(mat)
    Z = cluster.hierarchy.linkage(cd, method='ward')
    dn = cluster.hierarchy.dendrogram(Z, ax=ax[i])
    ax[i].set_xticks([])
    ax[i].set_title('%d-cluster consensus matrix' % (i + 2))

In [ ]:
silhouettes = np.zeros((4,3))
for i in range(len(consensus_mats)):
    mat = np.copy(consensus_mats[i])
    mat = 1 - mat
    np.fill_diagonal(mat, 0)
    cd = distance.squareform(mat)
    Z = cluster.hierarchy.linkage(cd, method='ward')
    for c in range(2, 5):
        clusters = cluster.hierarchy.fcluster(Z, c, criterion='maxclust')
        sil = metrics.silhouette_score(dis, clusters, metric='precomputed')
        silhouettes[i,c - 2] = sil

In [ ]:
silhouettes

In [ ]:
i, c = np.unravel_index(np.argmax(silhouettes, axis=None), silhouettes.shape) # return cell that has highest silhouette score
mat = np.copy(consensus_mats[i])
mat = 1 - mat # make into distance matrix
np.fill_diagonal(mat, 0)
cd = distance.squareform(mat)
Z = cluster.hierarchy.linkage(cd, method='ward')
clusters = cluster.hierarchy.fcluster(Z, c + 2, criterion='maxclust')

In [ ]:
np.unique(clusters, return_counts=True)

In [ ]:
pheno_c = []
for i in np.unique(clusters):
    pheno_c.append(pheno_et.iloc[clusters == i,:-1])

In [ ]:
pvals = np.zeros((np.unique(clusters).size, np.unique(clusters).size, 4))
for i in range(np.unique(clusters).size - 1):
    for j in range(i + 1, np.unique(clusters).size):
        for k in range(4):
            p = ttest_ind(pheno_c[i].iloc[:,k], pheno_c[j].iloc[:,k], equal_var=False)[1]
            pvals[i,j,k] = p
            pvals[j,i,k] = p

In [ ]:
p_corr = np.zeros(pvals.shape)
nmu = np.triu_indices(pvals.shape[0], 1)
for i in range(pvals.shape[2]):
    nm = np.triu_indices(np.unique(clusters).size, 1)
    p_unc = pvals[...,i][nm[0],nm[1]]
    res = multipletests(p_unc, method='fdr_bh')
    p_corr[nmu[0],nmu[1],i] = res[1]
    p_corr[...,i] += p_corr[...,i].T

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(11,11))
ax_flat = ax.flat
scores = pheno_et.columns.tolist()[:-1]
for i in range(p_corr.shape[2]):
    im = ax_flat[i].imshow(p_corr[...,i], cmap='jet', vmax=0.05)
    ax_flat[i].set_title(scores[i], fontsize=18)
    ax_flat[i].set_xticks(np.arange(p_corr.shape[0]))
    ax_flat[i].set_yticks(np.arange(p_corr.shape[0]))
    ax_flat[i].set_xticklabels(np.arange(p_corr.shape[0]) + 1)
    ax_flat[i].set_yticklabels(np.arange(p_corr.shape[0]) + 1)
cbar = fig.colorbar(im, ax=ax.ravel().tolist(), shrink=0.95)